In [ ]:
import requests
import pandas as pd
from datetime import timedelta, datetime
import time
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv('U1106683827_20230103_20240602_buy.csv')

df.head(5)
print(df.columns)



In [ ]:
df.head(6)

In [ ]:
df['Time(GMT+8)'] = pd.to_datetime(df['Time(GMT+8)'])
df = df.sort_values(by='Time(GMT+8)', ascending=True)
df.head()

In [ ]:
df = df.reset_index(drop = True)
df['id_transaction'] = df.index + 1
df.head(5)

In [ ]:
print(df['Items'].head(10))


In [ ]:
wear = ['Battle-Scarred', 'Factory New', 'Minimal Wear', 'Field-Tested', 'Well-Worn']
regex_pattern = f"({'|'.join(wear)})"
df['wear'] = df['Items'].str.extract(regex_pattern, expand=False)
df.head(5)



In [ ]:
df = df.rename(columns={'Time(GMT+8)' : 'Time'})
df.head(5)

In [ ]:
df['store_link'] = df['Items'].str.extract(r'HYPERLINK\("([^"]+)"')
df['item'] = df['Items'].str.extract(r',\s*"([^"]+)"\)')
df = df.drop(columns=['Items'])
df.head(5)

In [ ]:
df['Price_CNY'] = df['Price'].str.replace('¥', '', regex = False).astype(float)
df = df.drop('Price', axis=1)
df.head(5)

In [ ]:
df['Timestamp_GMT8'] = pd.to_datetime(df['Time'])
df['Transaction_Date'] = df['Time'].dt.date
df = df.drop('Time', axis= 1)
df.head(5)

In [ ]:
frankfurter_cache = {}

def buscar_cotacao_frankfurter(data_cotacao):

    data_str = data_cotacao.strftime('%Y-%m-%d')

    if data_str in frankfurter_cache:
        return frankfurter_cache[data_str]

    url = f"https://api.frankfurter.app/{data_str}?from=CNY&to=USD"

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        dados = response.json()

        if 'rates' in dados and 'USD' in dados['rates']:
            cotacao = dados['rates']['USD']
            frankfurter_cache[data_str] = cotacao
            return cotacao
        else:
            print(f"Não encontrou cotação USD para {data_str} na resposta: {dados}")
            frankfurter_cache[data_str] = None
            return None

    except requests.exceptions.RequestException as e:
        print(f"Erro de API/Rede ao buscar Frankfurter para {data_str}: {e}")
        frankfurter_cache[data_str] = None
        return None

print("Função 'buscar_cotacao_frankfurter' definida.")

In [ ]:
if 'Transaction_Date' not in df.columns:
    print("ERRO")
else:
    datas_unicas = df['Transaction_Date'].unique()

    cotacoes_map_usd = {data: buscar_cotacao_frankfurter(data) for data in datas_unicas if data}

    df['Taxa_Cambio_USD'] = df['Transaction_Date'].map(cotacoes_map_usd)

    df['Price_USD'] = df['Price_CNY'] * df['Taxa_Cambio_USD']

df = df.drop(columns=['BRL','Timestamp_GMT8', 'Taxa_Cambio_USD'])

df.head(5)